# BackTrader

In [1]:
from __future__ import (absolute_import, division, print_function,unicode_literals)
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
import numpy as np
import pandas as pd
# 画图
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec # Alignments
import seaborn as sns # theme & dataset
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.sans-serif']=['FangSong']
plt.rcParams['axes.unicode_minus']=False

import backtrader as bt

import tushare as ts
token = 'd9645bfd8516b93f1312d8ba696c83b606a78d966e71ec5c1e79bef4'
ts.set_token(token) 
pro = ts.pro_api(token)

In [2]:
# 使用Tushare获取数据，要严格保持OHLC的格式
df = ts.pro_bar(ts_code='600276.SH', adj='qfq',start_date='20160101', end_date='20210906')
df = df[['trade_date', 'open', 'high', 'low', 'close','vol']]
df.trade_date = pd.to_datetime(df.trade_date)
# 索引必须是日期
df.index = df.trade_date
# 日期必须要升序
df.sort_index(inplace=True)
df

HTTPConnectionPool(host='api.waditu.com', port=80): Read timed out. (read timeout=30)


,trade_date,open,high,low,close,vol
trade_date,,,,,,
2016-01-04,2016-01-04,14.9747,14.9747,14.3587,14.3953,86744.76
2016-01-05,2016-01-05,14.1818,14.6393,14.1665,14.5051,95328.66
2016-01-06,2016-01-06,14.5051,14.9076,14.4563,14.7979,86670.45
2016-01-07,2016-01-07,14.7155,14.7155,13.9225,14.0872,15488.00
2016-01-08,2016-01-08,14.4258,14.6362,14.0293,14.5325,112455.96
...,...,...,...,...,...,...
2021-08-31,2021-08-31,45.4500,46.1400,44.3300,45.2000,495741.85
2021-09-01,2021-09-01,45.4000,47.2000,44.9200,46.5900,731270.61
2021-09-02,2021-09-02,46.6000,46.6000,45.5000,45.8600,465752.10


## 策略：3天价格连续下跌买入，持股5天后卖出

In [3]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    # 设定参数，便于修改
    params = (
        ('exitbars', 5),
    )

    def log(self, txt, dt=None):
        ''' 
        日志函数：打印结果
        datas[0]：传入的数据，包含日期、OHLC等数据
        datas[0].datetime.date(0)：调用传入数据中的日期列
        '''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # dataclose变量：跟踪当前收盘价
        self.dataclose = self.datas[0].close

        # order变量：跟踪订单状态
        self.order = None
        # buyprice变量：买入价格
        self.buyprice = None
        # buycomm变量：买入时佣金费用
        self.buycomm = None

    def notify_order(self, order):
        '''订单状态通知(order.status)：提示成交状态'''
        if order.status in [order.Submitted, order.Accepted]:
            # 如果订单只是提交状态，那么啥也不提示
            return

        # 检查订单是否执行完毕
        # 注意：如果剩余现金不足，则会被拒绝！
        if order.status in [order.Completed]:
            if order.isbuy():
                # 买入信号记录：买入价、买入费用、买入佣金费用
                self.log(
                        'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                        (order.executed.price,
                         order.executed.value,
                         order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            elif order.issell():
                # 卖出信号记录：卖出价、卖出费用、卖出佣金费用
                self.log(
                        'SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                        (order.executed.price,
                         order.executed.value,
                         order.executed.comm))
            
            # 记录订单执行的价格柱的编号（即长度）
            self.bar_executed = len(self)

        # 如果订单被取消/保证金不足/被拒绝
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # 如果没有查询到订单，则订单为None
        self.order = None

    def notify_trade(self, trade):
        '''交易状态通知：查看交易毛/净利润'''
        if not trade.isclosed:
            return
        # 交易毛利润：trade.pnl、交易净利润：trade.pnlcomm（扣除佣金）
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                    (trade.pnl, trade.pnlcomm))


    # 交易策略主函数
    def next(self):
        # 记录收盘价
        self.log('Close, %.2f' % self.dataclose[0])

        # 检查一下是否有订单被挂起，如果有的话就先不下单
        if self.order:
            return
        
        # 检查一下目前是否持有头寸，如果没有就建仓
        # 策略：价格连续跌3天就买
        if not self.position:

            # 如果现价<昨日价格
            if self.dataclose[0] < self.dataclose[-1]:
                # 如果昨日价格<前日价格
                if self.dataclose[-1] < self.dataclose[-2]:
                    # 买买买！先记录一下买入价格（收盘价）
                    self.log('BUY CREATE, %.2f' % self.dataclose[0])
                    # 更新订单状态：buy()：开仓买入，买入价是下一个数据，即【开盘价】
                    self.order = self.buy()
        else:
             # 如果已经建仓，并持有头寸，则执行卖出指令
             # 如果当前价格柱已经距离执行订单过去5个，则卖出
             # 这里的【self.params.exitbars：5】可以通过最开始的params中的参数进行修改
            if len(self) >= (self.bar_executed + self.params.exitbars):
                # 卖!卖!卖!
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                # 更新订单状态：sell()：平仓卖出，卖出价是下一个数据，即【开盘价】
                self.order = self.sell()

# 创建实例
cerebro = bt.Cerebro()

 # 添加策略
cerebro.addstrategy(TestStrategy)

# 添加数据源
data = bt.feeds.PandasData(dataname=df)

# 输入数据源
cerebro.adddata(data)

# 设置初始现金：10万
cerebro.broker.setcash(100000.0)

# 设定每次买入的股票数量：10股
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# 设置佣金费率：双边0.1%
cerebro.broker.setcommission(commission=0.001)

# 显示：初始资产
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# 运行策略
cerebro.run()

# 显示：最终资产
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2016-01-04, Close, 14.40
2016-01-05, Close, 14.51
2016-01-06, Close, 14.80
2016-01-07, Close, 14.09
2016-01-08, Close, 14.53
2016-01-11, Close, 13.89
2016-01-12, Close, 13.74
2016-01-12, BUY CREATE, 13.74
2016-01-13, BUY EXECUTED, Price: 13.75, Cost: 13754.80, Comm 13.75
2016-01-13, Close, 13.40
2016-01-14, Close, 13.52
2016-01-15, Close, 13.12
2016-01-18, Close, 13.23
2016-01-19, Close, 13.52
2016-01-20, Close, 13.52
2016-01-20, SELL CREATE, 13.52
2016-01-21, SELL EXECUTED, Price: 13.39, Cost: 13754.80, Comm 13.39
2016-01-21, OPERATION PROFIT, GROSS -366.00, NET -393.14
2016-01-21, Close, 13.08
2016-01-21, BUY CREATE, 13.08
2016-01-22, BUY EXECUTED, Price: 13.20, Cost: 13202.80, Comm 13.20
2016-01-22, Close, 13.21
2016-01-25, Close, 13.41
2016-01-26, Close, 13.09
2016-01-27, Close, 13.22
2016-01-28, Close, 12.98
2016-01-29, Close, 13.42
2016-01-29, SELL CREATE, 13.42
2016-02-01, SELL EXECUTED, Price: 13.41, Cost: 13202.80, Comm 13.41
2016-02-01, OPE

## 策略：均线策略

In [4]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    # 设定参数，便于修改
    params = (
        ('maperiod', 60),
    )

    def log(self, txt, dt=None):
        ''' 
        日志函数：打印结果
        datas[0]：传入的数据，包含日期、OHLC等数据
        datas[0].datetime.date(0)：调用传入数据中的日期列
        '''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # dataclose变量：跟踪当前收盘价
        self.dataclose = self.datas[0].close

        # order变量：跟踪订单状态
        self.order = None
        # buyprice变量：买入价格
        self.buyprice = None
        # buycomm变量：买入时佣金费用
        self.buycomm = None

        # 指标：简单移动平均 MovingAverageSimple【15天】
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], 
                                                     period=self.params.maperiod)
        
        # 添加画图专用指标
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25,subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)

    def notify_order(self, order):
        '''订单状态通知(order.status)：提示成交状态'''
        if order.status in [order.Submitted, order.Accepted]:
            # 如果订单只是提交状态，那么啥也不提示
            return

        # 检查订单是否执行完毕
        # 注意：如果剩余现金不足，则会被拒绝！
        if order.status in [order.Completed]:
            if order.isbuy():
                # 买入信号记录：买入价、买入费用、买入佣金费用
                self.log(
                        'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                        (order.executed.price,
                         order.executed.value,
                         order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            elif order.issell():
                # 卖出信号记录：卖出价、卖出费用、卖出佣金费用
                self.log(
                        'SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                        (order.executed.price,
                         order.executed.value,
                         order.executed.comm))
            
            # 记录订单执行的价格柱的编号（即长度）
            self.bar_executed = len(self)

        # 如果订单被取消/保证金不足/被拒绝
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # 如果没有查询到订单，则订单为None
        self.order = None

    def notify_trade(self, trade):
        '''交易状态通知：查看交易毛/净利润'''
        if not trade.isclosed:
            return
        # 交易毛利润：trade.pnl、交易净利润：trade.pnlcomm（扣除佣金）
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                    (trade.pnl, trade.pnlcomm))


    # 交易策略主函数
    def next(self):
        # 记录收盘价
        self.log('Close, %.2f' % self.dataclose[0])

        # 检查一下是否有订单被挂起，如果有的话就先不下单
        if self.order:
            return
        
        # 检查一下目前是否持有头寸，如果没有就建仓
        # 策略：价格连续跌3天就买
        if not self.position:

            # 如果现价>【15日】均线
            if self.dataclose[0] > self.sma[0]:
                # 买买买！先记录一下买入价格（收盘价）
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                # 更新订单状态：buy()：开仓买入，买入价是下一个数据，即【开盘价】
                self.order = self.buy()
        else:
             # 如果已经建仓，并持有头寸，则执行卖出指令
             # 如果现价<【15日】均线
            if self.dataclose[0] < self.sma[0]:
                # 卖!卖!卖!
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                # 更新订单状态：sell()：平仓卖出，卖出价是下一个数据，即【开盘价】
                self.order = self.sell()

# 创建实例
cerebro = bt.Cerebro()

 # 添加策略
cerebro.addstrategy(TestStrategy)

# 添加数据源
data = bt.feeds.PandasData(dataname=df)

# 输入数据源
cerebro.adddata(data)

# 设置初始现金：10万
cerebro.broker.setcash(100000.0)

# 设定每次买入的股票数量：10股
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# 设置佣金费率：双边0.1%
cerebro.broker.setcommission(commission=0.001)

# 显示：初始资产
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# 运行策略
cerebro.run()

# 显示：最终资产
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

# 画出图像
cerebro.plot(volume=False)

Starting Portfolio Value: 100000.00
2016-04-01, Close, 14.50
2016-04-01, BUY CREATE, 14.50
2016-04-05, BUY EXECUTED, Price: 14.40, Cost: 14395.30, Comm 14.40
2016-04-05, Close, 14.61
2016-04-06, Close, 14.51
2016-04-07, Close, 14.55
2016-04-08, Close, 14.44
2016-04-11, Close, 14.34
2016-04-13, Close, 14.37
2016-04-14, Close, 14.40
2016-04-15, Close, 14.53
2016-04-18, Close, 14.68
2016-04-19, Close, 14.57
2016-04-20, Close, 14.59
2016-04-21, Close, 14.26
2016-04-22, Close, 14.29
2016-04-25, Close, 14.19
2016-04-26, Close, 14.44
2016-04-27, Close, 14.43
2016-04-28, Close, 14.26
2016-04-29, Close, 14.24
2016-04-29, SELL CREATE, 14.24
2016-05-03, SELL EXECUTED, Price: 14.30, Cost: 14395.30, Comm 14.30
2016-05-03, OPERATION PROFIT, GROSS -97.60, NET -126.29
2016-05-03, Close, 14.58
2016-05-03, BUY CREATE, 14.58
2016-05-04, BUY EXECUTED, Price: 14.55, Cost: 14547.80, Comm 14.55
2016-05-04, Close, 14.58
2016-05-05, Close, 14.50
2016-05-06, Close, 14.31
2016-05-06, SELL CREATE, 14.31
2016-05-0

<IPython.core.display.Javascript object>

[[<Figure size 600x400 with 7 Axes>]]

## 策略优化：以调整SMA参数为例

In [7]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
        ('printlog', False),
    )

    def log(self, txt, dt=None, doprint=False):
        # 参数：printlog、doprint，初始均设置为False，参数printlog控制细节结果输出
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None

        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' % (trade.pnl, trade.pnlcomm))

    def next(self):
        self.log('Close, %.2f' % self.dataclose[0])

        if self.order:
            return

        if not self.position:

            if self.dataclose[0] > self.sma[0]:
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
                self.order = self.sell()

    def stop(self):
        # 参数doprint=True，控制log函数打印以下结果
        self.log('(MA Period %2d) Ending Value %.2f' % (self.params.maperiod, self.broker.getvalue()), doprint=True)


# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
strats = cerebro.optstrategy(TestStrategy,maperiod=range(10, 120, 5))

# 添加数据源
data = bt.feeds.PandasData(dataname=df)

# 输入数据源
cerebro.adddata(data)

# Set our desired cash start
cerebro.broker.setcash(1000.0)

# Add a FixedSize sizer according to the stake
cerebro.addsizer(bt.sizers.FixedSize, stake=10)

# Set the commission
cerebro.broker.setcommission(commission=0.0)

# Run over everything
cerebro.run(maxcpus=1)

2021-09-06, (MA Period 10) Ending Value 875.79
2021-09-06, (MA Period 15) Ending Value 1089.22
2021-09-06, (MA Period 20) Ending Value 1135.87
2021-09-06, (MA Period 25) Ending Value 1100.01
2021-09-06, (MA Period 30) Ending Value 1225.00
2021-09-06, (MA Period 35) Ending Value 1346.81
2021-09-06, (MA Period 40) Ending Value 1263.91
2021-09-06, (MA Period 45) Ending Value 1241.21
2021-09-06, (MA Period 50) Ending Value 1275.95
2021-09-06, (MA Period 55) Ending Value 1276.11
2021-09-06, (MA Period 60) Ending Value 1274.00
2021-09-06, (MA Period 65) Ending Value 1317.69
2021-09-06, (MA Period 70) Ending Value 1341.51
2021-09-06, (MA Period 75) Ending Value 1372.77
2021-09-06, (MA Period 80) Ending Value 1360.12
2021-09-06, (MA Period 85) Ending Value 1430.61
2021-09-06, (MA Period 90) Ending Value 1369.22
2021-09-06, (MA Period 95) Ending Value 1385.95
2021-09-06, (MA Period 100) Ending Value 1424.73
2021-09-06, (MA Period 105) Ending Value 1438.00
2021-09-06, (MA Period 110) Ending Valu

[[<backtrader.cerebro.OptReturn at 0x1f54047b6c8>],